In [1]:
import warnings
warnings.filterwarnings("ignore")
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


학습 데이터를 만든다.

In [2]:
xData = [1, 2, 3, 4, 5, 6, 7] # 근무 시간
yData = [25000, 55000, 75000, 110000, 128000, 155000, 180000] # 근무 시간에 따른 매출 금액, 실제값

난수를 발생시켜 학습할 데이터의 기울기(가중치)와 y절편(바이어스)을 정한다.  
난수를 발생시켜서 작업하는 이유는 과적합을 방지하기 위해서이다.  
과적합이란 학습시킨 데이터에서는 높은 정확도를 보이지만 학습시킨 데이터 이외의 데이터에서는 낮은 정확도를 보이는 문제점을 말한다.

In [3]:
# Variable() : tensorflow에서 사용할 변수를 선언한다.
# tf.random_uniform([난수의 개수], 난수의 최소값, 난수의 최대값) : tensorflow에서 사용할 난수를 발생시킨다.
a = tf.Variable(tf.random_uniform([1], -100, 100)) # 기울기(가중치), -100 ~ 100 사이의 난수
b = tf.Variable(tf.random_uniform([1], -100, 100)) # y절편(바이어스), -100 ~ 100 사이의 난수
# tensorflow 변수는 아래와 같이 그냥 사용하면 저장된 데이터가 보이지 않고 tensorflow 객체가 출력된다.
print('a = {}, b = {}'.format(a, b))

a = <tf.Variable 'Variable:0' shape=(1,) dtype=float32_ref>, b = <tf.Variable 'Variable_1:0' shape=(1,) dtype=float32_ref>


tensorflow에서 사용할 변수를 초기화 하고 난수로 발생키킨 기울기와 y절편 값을 확인한다.

In [4]:
# tensorflow 변수에 저장된 데이터를 출력하려면 tensorflow의 Session을 만들고 변수를 초기화 시킨 후 run() 메소드를 사용해야
# 한다.
# tensorflow 세션을 만든다. => 세션에서 실행한다.
sess = tf.Session()
# global_variables_initializer() 메소드로 tensorflow 변수를 초기화 시킨다. => 변수의 값을 수정하고 연산을 다시 실행하려면
# 변수를 다시 초기화 시켜야 한다.
# init = tf.global_variables_initializer()
# tensorflow 세션에서 run() 메소드로 변수 초기화를 적용시킨다.
# sess.run(init)
sess.run(tf.global_variables_initializer())
print('a = {}, b = {}'.format(sess.run(a), sess.run(b)))

a = [-14.44323], b = [-69.455765]


근무 시간과 매출 금액을 기억할 tensorflow에서 사용하는 자료형(placeholder)을 선언한다.

In [5]:
x = tf.placeholder(dtype=tf.float32) # 근무 시간을 기억하는 placeholder
y = tf.placeholder(dtype=tf.float32) # 매출 금액을 기억하는 placeholder

1차 방정식 형태의 가설을 세우고 비용(오차) 함수를 정의한다.

In [6]:
# 1차 방정식 형태의 가설, Y => 예측값
Y = a * x + b
# 비용(오차) 함수 => 실제값(yData => y라는 placeholder에 저장한다.)과 예측값(Y)의 편차에 제곱에 대한 평균을 이용한다.
# => 평균 제곱법
# reduce_mean() : tensorflow에서 평균을 계산하는 메소드
# square() : tensorflow에서 제곱값을 계산하는 메소드
cost = tf.reduce_mean(tf.square(Y - y)) # 예측값과 실제값의 편차의 제곱에 대한 평균을 계산한다.

tensorflow의 경사 하강법 메소드로 비용(오차) 함수의 결과를 가장 작게 만드는 방향으로 학습을 시키도록 정의한다.

In [7]:
# 경사 하강 알고리즘의 학습율을 설정한다.
learning_rate = tf.Variable(0.01)
# tf.train.GradientDescentOptimizer() : tensorflow에서 학습율에 따른 비용(오차) 함수를 계산하는 함수
# optimizer = tf.train.GradientDescentOptimizer(learning_rate)
# minimize() : 경사 하강 알고리즘에 따라 인수로 지정된 비용(오차) 함수의 최소값을 찾는다.
# train = optimizer.minimize(cost)
train = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [13]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(5001):
    # 비용(오차) 함수를 가장 작게 만드는 학습을 할 수 있도록 placeholder에 데이터를 대입한다.
    # 학습할 데이터는 반드시 'feed_dict'라는 이름의 딕셔너리 타입의 자료형을 만들어 대입해야 한다.
    # feed_dict 딕셔너리의 key에는 placeholder의 이름을, value에는 각 placeholder에 대입할 리스트 타입의 데이터를 저장한다.
    
    # tensorflow 세션에서 비용(오차) 함수를 가장 작게 만드는 방샹으로 feed_dict 딕셔너리의 데이터로 학습시킨다.
    sess.run(train, feed_dict={x: xData, y: yData})
    
    # 500번 단위로 학습이 될 때 마다 머신러닝이 실행되는 중간 결과를 출력한다.
    if i % 500 == 0:
        # i, 오차, 기울기, y절편
        print('{0:4d} {1:13.2f} {2:10.2f} {3:10.2f}'.format(i, sess.run(cost, feed_dict={x: xData, y: yData}), 
                                                            sess.run(a)[0], sess.run(b)[0]))

   0 4622275072.00   10326.02    2072.34
 500    8687418.00   25537.20    1951.77
1000    8633814.00   25627.46    1504.82
1500    8632682.00   25640.61    1439.68
2000    8632655.00   25642.53    1430.19
2500    8632649.00   25642.80    1428.82
3000    8632656.00   25642.84    1428.63
3500    8632657.00   25642.85    1428.61
4000    8632657.00   25642.85    1428.61
4500    8632657.00   25642.85    1428.61
5000    8632657.00   25642.85    1428.61


학습이 완료된 후 원하는 입력에 대한 결과를 출력한다.

In [17]:
for i in range(8, 25):
    print('{0:2d}시간 근무를 했을 때 하루 매출 기대금액 : {1:8,.0f}'.format(i, sess.run(Y, feed_dict={x: [i]})[0]))

 8시간 근무를 했을 때 하루 매출 기대금액 :  206,571
 9시간 근무를 했을 때 하루 매출 기대금액 :  232,214
10시간 근무를 했을 때 하루 매출 기대금액 :  257,857
11시간 근무를 했을 때 하루 매출 기대금액 :  283,500
12시간 근무를 했을 때 하루 매출 기대금액 :  309,143
13시간 근무를 했을 때 하루 매출 기대금액 :  334,786
14시간 근무를 했을 때 하루 매출 기대금액 :  360,428
15시간 근무를 했을 때 하루 매출 기대금액 :  386,071
16시간 근무를 했을 때 하루 매출 기대금액 :  411,714
17시간 근무를 했을 때 하루 매출 기대금액 :  437,357
18시간 근무를 했을 때 하루 매출 기대금액 :  463,000
19시간 근무를 했을 때 하루 매출 기대금액 :  488,643
20시간 근무를 했을 때 하루 매출 기대금액 :  514,286
21시간 근무를 했을 때 하루 매출 기대금액 :  539,928
22시간 근무를 했을 때 하루 매출 기대금액 :  565,571
23시간 근무를 했을 때 하루 매출 기대금액 :  591,214
24시간 근무를 했을 때 하루 매출 기대금액 :  616,857
